In [1]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import scipy.special
import glob

pd.set_option('display.max_columns',85)
pd.set_option('display.max_rows',100)

In [2]:
# read in data to panda
# get the number of unclassified reads from Kraken2 outputs

InPath="/home/isabel/Documents/postDoc_Amsterdam/1_EvolWormJourney/1_Genomics/2_EvolGenomics/2_Metagenomics/1_taxonomy/Bracken/withDB_standard/"
#InPath="/home/isabel/Documents/postDoc_Amsterdam/1_EvolWormJourney/1_Genomics/2_EvolGenomics/2_Metagenomics/1_taxonomy/Bracken/withDB_plusPf/"
files=glob.glob(InPath+"*IR.genus.bracken")

OutName = "KraBracken_DBstandard_042024_0minFrac_genus"
OutPath="/home/isabel/Documents/postDoc_Amsterdam/1_EvolWormJourney/1_Genomics/2_EvolGenomics/2_Metagenomics/1_taxonomy/"

threshold = 0

In [3]:
# run a loop over all samples:
## -- extract the sample name from the file name 
## -- extract the data column wise
## -- merge to already existing df


for i in range(len(files)):

    # get sample name
    a = files[i].split('/')
    sample=a[-1].split('_')[0:5]
    sampleName='_'.join(sample)
    column = sampleName + "_abund"

    
    df_pre=pd.read_csv(files[i],header=0,delimiter="\t")
    #print(df_pre.shape)
    genus_here = df_pre['name']
    abund_here = df_pre['fraction_total_reads']

    d = {
        "genus": genus_here,
        column: abund_here
        }
    df_temp = pd.DataFrame(d)

    if i==0:
        df_merge = df_temp
    else:
        df_merge = pd.merge(df_merge, df_temp, on="genus", how="outer")

#threshold = 0.01
df_merge
df_collect = df_merge.fillna(0)

print(df_collect.shape)


(1843, 94)


In [4]:
# kick out rows that only contain 0:
## do this by first adding a column that includes the sum of samples that have this genus

rowNum = df_collect.shape[0]

sumSamples = []
for i in range(rowNum):
    a  = np.array(df_collect.iloc[i])[1:].astype(float)
    #print(a)
    aa = np.count_nonzero(a > threshold)
    sumSamples.append(aa)

sumSamples = np.array(sumSamples)

## and add it as column to the data frame:
#df.insert(0, "sumSamples", sumSamples)
#df

sumSamples
df_collect

genus  VU_bn50_0_3_woCE.IR.genus.bracken_abund  \
0          Abyssalbus                                  0.00000   
1          Abyssibius                                  0.00001   
2       Acaryochloris                                  0.00000   
3         Acetivibrio                                  0.00000   
4         Acetobacter                                  0.00002   
...               ...                                      ...   
1838         Zobellia                                  0.00000   
1839  Zophobihabitans                                  0.00000   
1840     Zunongwangia                                  0.00000   
1841       Zymobacter                                  0.00000   
1842        Zymomonas                                  0.00000   

      VU_bn50_15_3_woCE.IR.genus.bracken_abund  \
0                                          0.0   
1                                          0.0   
2                                          0.0   
3                                          0.0   
4                                          0.0   
...                                        ...   
1838                                       0.0   
1839                                       0.0   
1840                                       0.0   
1841                                       0.0   
1842                                       0.0   

      UVA_bb_0_1_woCE.IR.genus.bracken_abund  \
0                                    0.00000   
1                                    0.00002   
2                                    0.00000   
3                                    0.00000   
4                                    0.00008   
...                                      ...   
1838                                 0.00000   
1839                                 0.00003   
1840                                 0.00000   
1841                                 0.00003   
1842                                 0.00004   

      NIOO_A00_1_3_woCE.IR.genus.bracken_abund  \
0                                      0.00000   
1                                      0.00000   
2                                      0.00000   
3                                      0.00000   
4                                      0.00001   
...                                        ...   
1838                                   0.00000   
1839                                   0.00000   
1840                                   0.00000   
1841                                   0.00000   
1842                                   0.00000   

      NIOO_M00_15_4_woCE.IR.genus.bracken_abund  \
0                                       0.00000   
1                                       0.00000   
2                                       0.00000   
3                                       0.00000   
4                                       0.00001   
...                                         ...   
1838                                    0.00000   
1839                                    0.00000   
1840                                    0.00000   
1841                                    0.00000   
1842                                    0.00000   

      VU_bn50_15_1_woCE.IR.genus.bracken_abund  \
0                                      0.00000   
1                                      0.00001   
2                                      0.00000   
3                                      0.00000   
4                                      0.00000   
...                                        ...   
1838                                   0.00000   
1839                                   0.00000   
1840                                   0.00000   
1841                                   0.00000   
1842                                   0.00000   

      VU_bn5_15_2_woCE.IR.genus.bracken_abund  \
0                                     0.00000   
1                                     0.00000   
2                                     0.00000   
3                                 

In [13]:
# now only keep the rows that have a non zero entry

dropIdx = np.where(sumSamples == 0)[0]

df_nonZero = df_collect.drop(dropIdx)
df_nonZero


df_nonZero.set_index('genus',inplace=True)

df_2 = df_nonZero.transpose()

df_2.shape



(93, 1249)

In [6]:
# also, get the sample names as colums

institution = []
treatment = []
week = []
rep = []
newRowName = []

rowNames = df_2.index.tolist()

for i in rowNames:
    nameList = i.split('_')
    institution.append(nameList[0])
    treatment.append(nameList[1])
    week.append(nameList[2])
    rep.append(nameList[3])

    newRowName.append(nameList[0] + "_" + nameList[1] + "_" + nameList[2] + "_" + nameList[3])

#add columns to the dataframe
df_final=df_2
df_final.insert(0, "institution", institution)
df_final.insert(1, "treatment", treatment)
df_final.insert(2, "week", week)
df_final.insert(2, "replicate", rep)
df_final.insert(0, "name_short", newRowName)


In [10]:
# Sort and tidy up the dataframe
df_final=df_final.sort_values(['institution','treatment','week','replicate'])
df_final.shape

(93, 1254)

In [8]:
df_final.to_csv(OutPath+OutName+"_FromPY.csv", index=False) #
#df2=pd.read_csv("data.csv")

# To Do:
- is there a better wa than the transpose one?
- get rid of the 'consensus_taxonomy' entry

In [9]:
df_final.shape

(93, 1254)